<a href="https://colab.research.google.com/github/HWMV/AIFFEL_Quest1/blob/master/Main_quest/Main_quest3/rock_scissor_paper_background.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Main_quest3
# rock_scissor_paper
# 온라인 6기 코어 최현우

* **루브릭**
1. 이미지 분류기 모델이 성공적으로 만들어졌는가?	학습과정이 정상적으로 수행되었으며, 학습 결과에 대한 그래프를 시각화(ex. train acc / train loss / val acc / val loss 등) 해 보았음
2. 오버피팅을 극복하기 위한 적절한 시도가 있었는가?	오버피팅 극복을 위하여 데이터셋의 다양성, 정규화 등을 2가지 이상 시도해보았음
3. 분류모델의 test accuracy가 기준 이상 높게 나왔는가?	85% 이상 도달하였음

In [1]:
# 라이브러리 호출
import tensorflow as tf
import numpy as np

print(tf.__version__)
print(np.__version__)

2.14.0
1.23.5


In [2]:
# 데이터 불러오기 (로컬, 구글드라이브 마운트)
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from PIL import Image
import glob
import os

In [4]:
# 데이터 불러오기2
BASE_PATH = "/content/drive/MyDrive/Main_quest3/"

In [5]:
# paper 이미지 resize 하기 (28x28)(좀 크게 늘려보는중)
def resize_images(img_path):
    images=glob.glob(img_path + "/*.jpg")

    print(len(images), " images to be resized.")

    # 28 x 28 size 로 resize (128 x 128로 해보자. 피처를 추출을 못하는 것 같아!)
    target_size=(64,64)
    for img in images:
        old_img=Image.open(img)
        new_img=old_img.resize(target_size,Image.ANTIALIAS)
        new_img.save(img, "JPEG")

    print(len(images), " images resized.")

# 함수 이용해서 가위바위보 이미지 전부 resize
resize_images(BASE_PATH + "rock")
resize_images(BASE_PATH + "scissor")
resize_images(BASE_PATH + "paper")

print("resize 완료!")

1247  images to be resized.


<ipython-input-5-246436236657>:11: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  new_img=old_img.resize(target_size,Image.ANTIALIAS)


1247  images resized.
1208  images to be resized.
1208  images resized.
1284  images to be resized.
1284  images resized.
resize 완료!


In [6]:
# load_data 함수 만들기 (노드 참조)

def load_data(img_path, number_of_data=3851):
    img_size=64
    color=3
    imgs=np.zeros(number_of_data*img_size*img_size*color,dtype=np.int32).reshape(number_of_data,img_size,img_size,color)
    labels=np.zeros(number_of_data,dtype=np.int32)

    idx=0
    for file in glob.iglob(img_path+'/scissor/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img
        labels[idx]=0
        idx=idx+1

    for file in glob.iglob(img_path+'/rock/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img
        labels[idx]=1
        idx=idx+1

    for file in glob.iglob(img_path+'/paper/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img
        labels[idx]=2
        idx=idx+1

    print("Number of images:", idx)
    return imgs, labels

In [7]:
# test_load_data 함수
def test_load_data(img_path, number_of_data=441):
    img_size=64
    color=3
    imgs=np.zeros(number_of_data*img_size*img_size*color,dtype=np.int32).reshape(number_of_data,img_size,img_size,color)
    labels=np.zeros(number_of_data,dtype=np.int32)

    idx=0
    for file in glob.iglob(img_path+'test/scissor/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img
        labels[idx]=0
        idx=idx+1

    for file in glob.iglob(img_path+'test/rock/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img
        labels[idx]=1
        idx=idx+1

    for file in glob.iglob(img_path+'test/paper/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img
        labels[idx]=2
        idx=idx+1

    print("Number of images:", idx)
    return imgs, labels

In [8]:
import os
print(os.listdir("/content/drive/MyDrive/Main_quest3/rock"))

['342.jpg', '127 (1).jpg', '1 (3).jpg', '246.jpg', '269.jpg', '400.jpg', '465.jpg', '233.jpg', '143 (1).jpg', '304.jpg', '408.jpg', '448.jpg', '18 (3).jpg', '204.jpg', '406.jpg', '21 (3).jpg', '4 (3).jpg', '454.jpg', '166 (1).jpg', '422.jpg', '327.jpg', '295.jpg', '197.jpg', '240.jpg', '264.jpg', '220.jpg', '524.jpg', '428.jpg', '374.jpg', '458.jpg', '335.jpg', '81 (3).jpg', '436.jpg', '355.jpg', '501.jpg', '134 (1).jpg', '189.jpg', '452.jpg', '347.jpg', '431.jpg', '369.jpg', '430.jpg', '417.jpg', '349.jpg', '471.jpg', '115 (1).jpg', '201.jpg', '370.jpg', '186.jpg', '236.jpg', '244.jpg', '85 (3).jpg', '396.jpg', '381.jpg', '439.jpg', '126 (1).jpg', '110 (1).jpg', '56 (3).jpg', '361.jpg', '287.jpg', '105 (2).jpg', '113 (1).jpg', '305.jpg', '389.jpg', '52 (3).jpg', '168 (1).jpg', '510.jpg', '6 (3).jpg', '486.jpg', '357.jpg', '412.jpg', '75 (3).jpg', '251.jpg', '445.jpg', '245.jpg', '124 (1).jpg', '25 (3).jpg', '483.jpg', '26 복사본.jpg', '62.jpg', '38 복사본 3.jpg', '49 복사본 2.jp

In [9]:
# 흑백 배경 만들기 위한 mask 이미지 생성

import cv2
import numpy as np
import sys

image_path = BASE_PATH + 'rock'
image = cv2.imread(image_path)

if image is None:
    print(f"Error loading image: {image_path}")
    sys.exit()

def generate_mask(image_path, save_path):
    """
    이미지에서 배경 마스크를 생성하고 저장하는 함수

    :param image_path: 마스크를 생성할 원본 이미지의 경로
    :param save_path: 생성된 마스크를 저장할 경로
    :return: None
    """

    # 이미지 불러오기
    image = cv2.imread(image_path)

    # 이미지를 그레이스케일로 변환
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # 이진화
    _, binary = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # 노이즈 제거를 위한 모폴로지 연산
    kernel = np.ones((5,5),np.uint8)
    cleaned = cv2.morphologyEx(binary, cv2.MORPH_OPEN, kernel)
    cleaned = cv2.morphologyEx(cleaned, cv2.MORPH_CLOSE, kernel)

    # 마스크 저장
    cv2.imwrite(save_path, cleaned)

def generate_mask_for_all_images(base_image_path, save_directory):
    """
    디렉터리 내의 모든 이미지에 대해 마스크를 생성하고 저장합니다.
    """
    if not os.path.exists(save_directory):
        os.makedirs(save_directory)

    # base_image_path 내의 모든 이미지 파일에 대해
    for file in os.listdir(base_image_path):
        if file.endswith('.jpg'):  # jpg 이미지만 처리
            image_path = os.path.join(base_image_path, file)
            mask_save_path = os.path.join(save_directory, file)

            generate_mask(image_path, mask_save_path)  # 위에서 정의한 함수 사용

# 사용
base_image_path = os.path.join(BASE_PATH, 'rock')
save_directory = os.path.join(BASE_PATH, 'Segmentation')
generate_mask_for_all_images(base_image_path, save_directory)

# 함수 사용
generate_mask(BASE_PATH + 'rock', BASE_PATH + 'Segmentation')

Error loading image: /content/drive/MyDrive/Main_quest3/rock


SystemExit: ignored

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
# 흑백 배경으로 데이터 변환 (Image Segmentation)
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate

def build_unet(input_shape):
    inputs = Input(input_shape)

    # Encoding (downsampling)
    c1 = Conv2D(16, (3, 3), activation='relu', padding='same')(inputs)
    p1 = MaxPooling2D((2, 2))(c1)

    c2 = Conv2D(32, (3, 3), activation='relu', padding='same')(p1)
    p2 = MaxPooling2D((2, 2))(c2)

    c3 = Conv2D(64, (3, 3), activation='relu', padding='same')(p2)
    p3 = MaxPooling2D((2, 2))(c3)

    # Bottleneck
    b = Conv2D(128, (3, 3), activation='relu', padding='same')(p3)

    # Decoding (upsampling)
    u1 = UpSampling2D((2, 2))(b)
    merge1 = concatenate([u1, c3])
    c4 = Conv2D(64, (3, 3), activation='relu', padding='same')(merge1)

    u2 = UpSampling2D((2, 2))(c4)
    merge2 = concatenate([u2, c2])
    c5 = Conv2D(32, (3, 3), activation='relu', padding='same')(merge2)

    u3 = UpSampling2D((2, 2))(c5)
    merge3 = concatenate([u3, c1])
    c6 = Conv2D(16, (3, 3), activation='relu', padding='same')(merge3)

    # Output layer
    outputs = Conv2D(1, (1, 1), activation='sigmoid')(c6)

    return tf.keras.Model(inputs=inputs, outputs=outputs)

# Build the U-Net model
input_shape = (128, 128, 3)  # Example input shape
model = build_unet(input_shape)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
# train_test_split 함수 불러오기
from sklearn.model_selection import train_test_split

# train data 설정
(x_data, y_data) = load_data(BASE_PATH)

# Train, Validation 데이터 분리
x_train, x_val, y_train, y_val = train_test_split(x_data, y_data, test_size=0.2, random_state=2023)

# 데이터 정규화
x_train_norm = x_train / 255.0
x_val_norm = x_val / 255.0

print("x_train shape: {}".format(x_train.shape))
print("y_train shape: {}".format(y_train.shape))
print("x_val shape: {}".format(x_val.shape))
print("y_val shape: {}".format(y_val.shape))

In [ ]:
# Load test data
# 위 과정 경로만 바꿔서 test data 세팅
resize_images(BASE_PATH + "test/rock")
resize_images(BASE_PATH + "test/scissor")
resize_images(BASE_PATH + "test/paper")
(x_test, y_test) = test_load_data(BASE_PATH)
x_test_norm = x_test / 255.0

print("x_test shape: {}".format(x_test.shape))
print("y_test shape: {}".format(y_test.shape))

In [ ]:
# 데이터들 확인
import matplotlib.pyplot as plt

plt.imshow(x_train[2])
print('라벨: ', y_train[2]) # 가위, 실제값 가위

In [ ]:
# 모델 구성하기 Sequestial 모델 사용
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# 데이터 증강(ImageDataGenerator() 활용)
datagen = ImageDataGenerator(
    rotation_range=10, # 이미지를 0~10도 사이로 랜덤하게 회전
    zoom_range=0.1, # 0.9~1.1 사이로 랜덤하게 확대/축소
    width_shift_range=0.1,  # 10% 범위에서 좌/우 이동
    height_shift_range=0.1, # 10% 범위에서 상/하 이동
    horizontal_flip=True # 좌우 반전
)
# train data가 너무 오버피팅(과적합) 되어서 처리 해줌
datagen.fit(x_train_norm)

# Model(Sequential) 구성 (뉴런 수 조절 : 오버피팅 방지)
model = Sequential()
# Layer 마다 뉴런 수 감소 : 오버피팅 방지
model.add(tf.keras.layers.Conv2D(128, (3,3), activation='relu', input_shape=(64,64,3)))
model.add(tf.keras.layers.MaxPooling2D(2,2))
model.add(tf.keras.layers.Conv2D(64, (3,3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D((2,2)))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(32, activation='relu'))

# 출력층
model.add(tf.keras.layers.Dense(3, activation='softmax'))

model.summary()

In [ ]:
# 모델 compile 하기
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Early stopping 추가
early_stopping = EarlyStopping(monitor='val_loss', patience=7)

# 학습 및 검증 실행
history = model.fit(datagen.flow(x_train_norm, y_train),
                    epochs=15,
                    validation_data=(x_val_norm, y_val),
                    callbacks=[early_stopping])

In [ ]:
# Model evaluation(Train)
train_loss, train_accuracy = model.evaluate(x_train_norm, y_train, verbose=2)
print("train_loss: {} ".format(train_loss))
print("train_accuracy: {}".format(train_accuracy))

# Model evaluation(Validation)
val_loss, val_accuracy = model.evaluate(x_val_norm, y_val, verbose=2)
print("val_loss: {} ".format(val_loss))
print("val_accuracy: {}".format(val_accuracy))

# Model evaluation(Test)
test_loss, test_accuracy = model.evaluate(x_test_norm, y_test, verbose=2)
print("test_loss: {} ".format(test_loss))
print("test_accuracy: {}".format(test_accuracy))

In [ ]:
# 시각화
import matplotlib.pyplot as plt

# Train and Validation Loss 그래프
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# Train and Validation Accuracy 그래프
plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
import random

# 테스트 데이터에서 무작위로 5개의 인덱스를 선택
random_indices = random.sample(range(x_test.shape[0]), 5)

# 해당 인덱스의 이미지와 라벨 가져오기
random_images = x_test[random_indices]
random_labels = y_test[random_indices]

# 이미지에 대한 예측값 계산
predictions = model.predict(random_images)

# 예측값에서 가장 높은 확률을 가진 클래스 선택
predicted_labels = np.argmax(predictions, axis=1)

# 이미지와 예측값, 실제값 비교하기
plt.figure(figsize=(15, 5))
for i, index in enumerate(random_indices):
    plt.subplot(1, 5, i + 1)
    plt.imshow(random_images[i])
    plt.title(f"Predicted: {predicted_labels[i]}, True: {random_labels[i]}")
    plt.axis('off')
plt.show()

# **회 고 록**
* ## train data의 오버피팅(과적합) 방지를 위한 방법
1. 데이터 shuffle 사용해보자 : 데이터 확장, 오버피팅 방지
shuffle_data = np.arange(x_train.shape[0])
np.random.shuffle(shuffle_data)

2. 데이터 증강(ImageDataGenerator(), fit에서 datagen.flow 활용)
했었으나 데이터 수를 늘리고 증강을 빼고 성능이 좋아서 삭제 했습니다 (충분히 train data로 학습)
데이터 수 회전,확대/축소 등으로 데이터 증강
(train data : 300장 > 1200장 > 3851장)
(validation data : train data 20%)
(test data : 639장)

3. 픽셀 정규화
x_data_norm = x_data / 255.0
x_train_norm = x_train / 255.0
x_val_norm = x_val / 255.0
x_test_norm = x_test / 255.0

4. dropout, BatchNormalization
각 layer 사이에 dropout(50%), BatchNormalization 적용
(train data가 너무 오버피팅 되서 조절)

---
* ## test data의 성능 향상을 위한 방법
1. epoch 수 증가 (10 > 30)
2. 데이터 셔플 제외 라벨 데이트까지 섞여서 확인 시 클래스가 섞여 나오는 것 확인
3. 쓸데없이 오버피팅 방지한 것들 제거 중 (dropout, batch normalization 등)
4. test data 비율 10~20% 정도로 줄여서 test


---

## **배우고 느낀 점**

test data를 더 늘리고, train data도 더 늘려서 작업해보고 싶지만 업로드와 epoch의 시간소요에 여기까지만 조절 했습니다

fit 학습을 여러번 반복하면 기존 학습에 이어서 학습을 해서 제대로 학습이 안되는 부분도 배웠습니다. 코드를 중복되지 않게 구현 하는 연습도 해봐야 할 것 같다고 생각 했습니다.

여러가지로 오버피팅 방지가 어렵고 관건이라고 배웠습니다. 그래도 실험해보면서 바뀌는 모습을 보면서 재밌게 작업했습니다. 여러 시도를 해보기 위해 컴퓨터 사양을 늘려야 효율적일 것 같다고 느꼈습니다.

오버피팅의 어려움을 체감하고 오버피팅(과적합) 방지에 여러 방법이 있고, 왜 사용하는지를 정확하게 이해하고 사용해야 된다는 것을 배웠습니다. 시각화의 중요성도 많이 배웠습니다.
더욱 정진 하겠습니다.

---

## **해결 못한 점**
1. pred값과 실제값 비교에서 흑백 사진의 정체
(해결 : train, test 데이터 load 하는 함수 따로 생성하고 데이터 수를 맞춰 줬습니다)
2. test 성능 향상을 마저 못했습니다
(가위 클래스 0은 잘 분석하나, 정면 바위 사진을 2(보)로 예측하는 경우가 많았습니다. 한 그루분의 test data가 굉장히 특이한 형태로 만드셨습니다)
(학습 데이터에서 특이한 바위와 보 데이터를 더 확보해야 될 것 같다고 분석했습니다)